In [13]:
import nest_asyncio
import twint
from models import *
from pymongo import MongoClient
from pycoingecko import CoinGeckoAPI
from datetime import datetime, timezone

### Conectar con MongoDB

In [14]:
mongo = MongoClient('2.tcp.ngrok.io:19533')

db = mongo.proyectoBigData

### Inicializar CoinGecko API Wrapper

In [15]:
cg = CoinGeckoAPI()

### Recuperar id, nombre y abreviación de las criptomonedas disponibles

In [16]:
criptocurrency_list = cg.get_coins_list()

db.criptomonedas.insert_many(criptocurrency_list)

### Recuperar historial de precios, capitalización de mercado y volumen de las criptomonedas seleccionadas

In [17]:
# Definimos los tokens a buscar
token_list = ["bitcoin", "dogecoin", "ethereum"]

# Fecha desde la que se comienza a buscar
from_date = datetime(2020, 1, 1)

days_ago = (datetime.now() - from_date).days

# intervalo entre cada recuperación de datos
time_interval = 'daily'

# moneda fíat con la que comparar el valor de las criptomonedas a buscar
moneda_comparacion = 'usd'

for token in token_list:
    
    # Recuperar historial de la criptomoneda
    historial = cg.get_coin_market_chart_by_id(id=token, vs_currency=moneda_comparacion,
                                               days=days_ago, interval=time_interval)
    
    # Histórico precios
    price_list = [vars(Precio(token, x[1], datetime.fromtimestamp(x[0] / 1000, timezone.utc)))
                  for x in  historial['prices']]
    
    db.priceHistory.insert_many(price_list)
    
    # Histórico de capitalización de mercado
    market_cap_list = [vars(MarketCap(token, x[1], datetime.fromtimestamp(x[0] / 1000, timezone.utc)))
                       for x in  historial['market_caps']]
    
    db.marketCapHistory.insert_many(market_cap_list)
    
    # Histórico de volumen de negociación
    volume_list = [vars(Volume(token, x[1], datetime.fromtimestamp(x[0] / 1000, timezone.utc)))
                   for x in  historial['total_volumes']]
    
    db.volumeHistory.insert_many(volume_list)

### Recuperar tweets relacionados a criptomonedas

In [18]:
nest_asyncio.apply()

# Lista de palabras a buscar
keywords = [
        "cripto",
        "criptomoneda",
        "criptomonedas",
        "crypto",
        "cryptocurrency",
        "cryptocurrencies",
        "bitcoin",
        "BTC",
        "$BTC",
        "ethereum",
        "ETH",
        "$ETH"
        "dogecoin",
        "DOGE",
        "$DOGE",
    ]

# Inicializamos el scraper de Twitter
config = twint.Config()

# Excluimos retweets de la búsqueda
config.Retweets = False

# Mínimo de likes que deba tener un tweet para ser recuperado
config.Min_likes = 5000

config.Hide_output = True
config.Store_object = True

lista_tweets = []

for word in keywords:

    # Seleccionamos la palabra clave a buscar
    config.Search = word
    
    # Realizamos la búsqueda
    twint.run.Search(config)
    
    # Recuperamos los resultados
    tweets = twint.output.tweets_list
    
    # Procesamos los resultados
    tweets_clean = [vars(Tweet(x.id, x.tweet, x.hashtags, x.cashtags,
                               datetime.strptime(x.datestamp + x.timestamp + x.timezone, '%Y-%m-%d%H:%M:%S%z'),
                               x.username, x.name, x.link, word)) for x in tweets]
    
    # Añadimos a la lista de resultados
    lista_tweets.extend(tweets_clean)

# Guardamos en mongo
db.tweetsCripto.insert_many(lista_tweets)

KeyboardInterrupt: 